In [1]:
# Import Data handling/display libraries
# Import Data handling/display libraries
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from typing import List, Union, Dict
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
# Import the Taiwan Credit Card Default training dataset
train = pd.read_csv('./input/Taiwan-Credit-Card-Default.csv')

# Drop the first row because it is a redundant heading row.
train.to_csv('./input/Taiwan-Credit-Card-New.csv', header=False, index=False)
# Re-read the dataset with its proper column headers
train = pd.read_csv('./input/Taiwan-Credit-Card-New.csv')
train.drop(["ID"], axis=1, inplace=True)
train.rename(columns={'default payment next month':'DEFAULT'}, inplace=True)
train.sample(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
23539,200000,2,1,1,40,0,0,0,0,0,...,10197,10118,10535,1138,1157,1149,500,1000,1000,0
25874,170000,2,1,1,35,2,2,2,2,2,...,139143,140987,144187,4800,2000,11000,5500,5500,5400,1
5406,50000,2,1,2,25,0,0,0,0,0,...,40655,40930,40146,2200,2000,2000,2000,3000,3000,0
4735,500000,1,2,1,48,-1,-1,-1,-1,-1,...,37985,33050,22185,35465,75212,38169,33209,22289,22289,0
15532,140000,1,1,1,39,1,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,1


In [3]:
class SelectCols(TransformerMixin):
    """Select columns from a DataFrame."""
    def __init__(self, cols: List[str]) -> None:
        self.cols = cols
    def fit(self, x: None) -> "SelectCols":
        """Nothing to do."""
        return self
    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        """Return just selected columns."""
        return x[self.cols]

In [4]:
class NumericEncoder(TransformerMixin):
    """Remove invalid values from numerical columns, replace with median."""
    def fit(self, x: pd.DataFrame) -> "NumericEncoder":
        """Learn median for every column in x."""
        # Find median for all columns, handling non-NaNs invalid values and NaNs
        # Where all values are NaNs (after coercion) the median value will be a NaN.
        self.encoders_ = {
            c: pd.to_numeric(x[c],
                             errors='coerce').median(skipna=True) for c in x}

        return self

    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        """For each column in x, encode NaN values are learned 
        median and add a flag column indicating where these 
        replacements were made"""

        # Create a list of new DataFrames, each with 2 columns
        output_dfs = []
        for c in x:
            new_cols = pd.DataFrame()
            # Find invalid values that aren't nans (-inf, inf, string)
            invalid_idx = pd.to_numeric(x[c].replace([-np.inf, np.inf],
                                                     np.nan),
                                        errors='coerce').isnull()

            # Copy to new df for this column
            new_cols.loc[:, c] = x[c].copy()
            # Replace the invalid values with learned median
            new_cols.loc[invalid_idx, c] = self.encoders_[c]
            # Mark these replacement in a new column called 
            # "[column_name]_invalid_flag"
            new_cols.loc[:, f"{c}_invalid_flag"] = invalid_idx.astype(np.int8)

            output_dfs.append(new_cols)

        # Concat list of output_dfs to single df
        df = pd.concat(output_dfs,
                       axis=1)

        # Return wtih an remaining NaNs removed. These might exist if the median
        # is a NaN because there was no numeric data in the column at all.
        return df.fillna(0)

In [5]:
## Constructing the pipeline
# NumericEncoding fork: Select numeric columns -> numeric encode
pp_numeric_cols = Pipeline([('select', SelectCols(cols=['LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE',
                                                       'PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6',
                                                       'BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                                                       'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6','DEFAULT'])),
                            ('process', NumericEncoder())])

In [8]:
# .fit_transform on the adult dataset
#columns = ["age","workclass","fnlwgt","education","educational-num","marital-status","occupation","relationship",
#           "race","gender","capital-gain","capital-loss","hours-per-week","native-country","income"]

train_pp = pp_numeric_cols.fit_transform(train)
print(train_pp.sample(5))
train_pp.to_csv('./input/Taiwan-Credit-Card-New.csv',index=False)

       LIMIT_BAL  LIMIT_BAL_invalid_flag  SEX  SEX_invalid_flag  EDUCATION  \
325     210000.0                       0  1.0                 0        2.0   
17351    20000.0                       0  2.0                 0        2.0   
16844   240000.0                       0  1.0                 0        1.0   
23443   260000.0                       0  2.0                 0        2.0   
5140    200000.0                       0  1.0                 0        1.0   

       EDUCATION_invalid_flag  MARRIAGE  MARRIAGE_invalid_flag   AGE  \
325                         0       2.0                      0  43.0   
17351                       0       1.0                      0  43.0   
16844                       0       1.0                      0  36.0   
23443                       0       1.0                      0  38.0   
5140                        0       1.0                      0  43.0   

       AGE_invalid_flag          ...           PAY_AMT3  \
325                   0          ...   